In [1]:
!jupyter kernelspec list

Available kernels:
  pyspark        C:\Anaconda3\envs\aiml\share\jupyter\kernels\PySpark
  pysparkauto    C:\Anaconda3\envs\aiml\share\jupyter\kernels\PySparkAuto
  python3        C:\Anaconda3\envs\aiml\share\jupyter\kernels\python3


In [3]:
%pycat C:\Anaconda3\envs\aiml\share\jupyter\kernels\PySpark\kernel.json
    

In [4]:
from pyspark.sql import *
import pyspark.sql.functions as F 


In [5]:
spark = SparkSession.builder.appName("basic").getOrCreate()

In [6]:
spark

In [11]:
len(dir(DataFrame)), len(dir(Column)), len(dir(F))
file = 'data/spark/Cartier+for+WinnersCurse.csv'
sdf = spark.read.format('csv').option("header", "true")\
                .option("inferSchema", "true").load(file)
sdf.show()

In [15]:
sdf.count()
sdf.rdd.getNumPartitions()

#ops
sdf.columns
sdf.dtypes
sdf.printSchema()

1

In [24]:
#filter
sdf.filter("bidderrate == 5")   #.show()
#or
sdf.filter(sdf.bidderrate == 5) #.show()
#or 
sdf.filter(sdf['bidderrate'] == 5) #.show()
#or 
sdf.filter(F.col('bidderrate') == 5)  #.show()  #This version

#display rows where bidder is "adammurry" or carloss8055
sdf.filter(F.col('bidder').isin("adammurry" , 'carloss8055') ).show()

+----------+------+-----------+-----------+----------+-------+-------+
| auctionid|   bid|    bidtime|     bidder|bidderrate|openbid|  price|
+----------+------+-----------+-----------+----------+-------+-------+
|1638843936|1500.0|5.226377315|carloss8055|         5|  500.0| 1625.0|
|1638843936|1550.0|6.892916667|carloss8055|         5|  500.0| 1625.0|
|1638843936|1625.0|6.893113426|carloss8055|         5|  500.0| 1625.0|
|1638844464| 450.0|6.736319444|  adammurry|         5|  300.0|  740.0|
|1638844464| 500.0|6.736469907|  adammurry|         5|  300.0|  740.0|
|1638844464| 700.0|6.995763889|  adammurry|         5|  300.0|  740.0|
|1638844464| 740.0| 6.99974537|  adammurry|         5|  300.0|  740.0|
|1640550476| 260.0|6.917569444|carloss8055|         5|    1.0|  300.0|
|1640550476| 280.0|6.980590278|carloss8055|         5|    1.0|  300.0|
|1641628327|1650.0|6.951226852|carloss8055|         5|  500.0|1894.78|
|1641628327|1700.0| 6.95150463|carloss8055|         5|  500.0|1894.78|
|16416

In [29]:
#Select - Transformations vs actions
sdf.select("auctionid")\
    .distinct()\
    .count()

97

In [32]:
sdf.select("auctionid")\
    .distinct().explain(extended=True)

== Parsed Logical Plan ==
Deduplicate [auctionid#10]
+- Project [auctionid#10]
   +- Relation[auctionid#10,bid#11,bidtime#12,bidder#13,bidderrate#14,openbid#15,price#16] csv

== Analyzed Logical Plan ==
auctionid: int
Deduplicate [auctionid#10]
+- Project [auctionid#10]
   +- Relation[auctionid#10,bid#11,bidtime#12,bidder#13,bidderrate#14,openbid#15,price#16] csv

== Optimized Logical Plan ==
Aggregate [auctionid#10], [auctionid#10]
+- Project [auctionid#10]
   +- Relation[auctionid#10,bid#11,bidtime#12,bidder#13,bidderrate#14,openbid#15,price#16] csv

== Physical Plan ==
*(2) HashAggregate(keys=[auctionid#10], functions=[], output=[auctionid#10])
+- Exchange hashpartitioning(auctionid#10, 200)
   +- *(1) HashAggregate(keys=[auctionid#10], functions=[], output=[auctionid#10])
      +- *(1) FileScan csv [auctionid#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/D:/handson/Module1/B2_24_25Sept/data/spark/Cartier+for+WinnersCurse.csv], PartitionFilters: [], PushedFilter

In [49]:
#creation of new column
df3 = sdf.withColumn("new_price", F.col("price") * 2)
#full table aggregation
sdf.agg(F.max("bid"), F.count("bidder")).show()
#per some column eg auctionid 
sdf.groupBy('auctionid').agg(F.sum('bid').alias("bids"), F.count('bidder').alias('cb'))\
   .select("auctionid",'bids', 'cb')

+--------+-------------+
|max(bid)|count(bidder)|
+--------+-------------+
|  5400.0|         1348|
+--------+-------------+



DataFrame[auctionid: int, bids: double, cb: bigint]

In [42]:
#stats 
sdf.summary().select('summary', 'price')  #.show()
#crosstab
sdf.stat.crosstab("bidder", "auctionid")  #.show()

DataFrame[bidder_auctionid: string, 1638843936: bigint, 1638844284: bigint, 1638844464: bigint, 1638844729: bigint, 1638917885: bigint, 1639226378: bigint, 1639253454: bigint, 1639309309: bigint, 1639323228: bigint, 1639333116: bigint, 1639341131: bigint, 1639364679: bigint, 1639425502: bigint, 1639672910: bigint, 1639929579: bigint, 1639979107: bigint, 1640179146: bigint, 1640550476: bigint, 1640653873: bigint, 1640793161: bigint, 1640936328: bigint, 1641062012: bigint, 1641242797: bigint, 1641457876: bigint, 1641587440: bigint, 1641628327: bigint, 1641629836: bigint, 1641722275: bigint, 1641783552: bigint, 1641880134: bigint, 1641880908: bigint, 1642005995: bigint, 1642185637: bigint, 1642514892: bigint, 1642534283: bigint, 1642561397: bigint, 1642875447: bigint, 1642876200: bigint, 1642911743: bigint, 1643136423: bigint, 1643187764: bigint, 1643201832: bigint, 1643212361: bigint, 1643244227: bigint, 1644046945: bigint, 1644049270: bigint, 1644077790: bigint, 1644077820: bigint, 1644

In [48]:
#sql
sdf.registerTempTable("auctions")
spark.sql("select count(*) as count from auctions").show()
#UDF
from pyspark.sql.types import *
slen = F.udf(lambda s: len(s), IntegerType())
spark.udf.register("slen", slen)
spark.sql("select slen(bidder) as len from auctions").count()

+-----+
|count|
+-----+
| 1348|
+-----+



1348

In [51]:
import pandas as pd
url = "https://raw.githubusercontent.com/ndas1971/Misc/master/bikes.csv"
df = pd.read_csv(url, index_col='Date', parse_dates=True, dayfirst=True)
df.describe()

,Berri1,CSC,Mais1,Mais2,Parc,PierDup,Rachel1,Totem_Laurier
count,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000,261.000000
mean,2743.390805,1221.858238,1757.590038,3224.130268,1669.425287,1152.885057,3084.425287,1858.793103
std,2247.957848,1070.037364,1458.793882,2589.514354,1363.738862,1208.848429,2380.255540,1434.899574
min,0.000000,0.000000,1.000000,0.000000,6.000000,0.000000,0.000000,0.000000
25%,392.000000,12.000000,236.000000,516.000000,222.000000,12.000000,451.000000,340.000000
50%,2771.000000,1184.000000,1706.000000,3178.000000,1584.000000,818.000000,3111.000000,2087.000000
75%,4767.000000,2168.000000,3158.000000,5812.000000,3068.000000,2104.000000,5338.000000,3168.000000
max,6803.000000,3330.000000,4716.000000,7684.000000,4103.000000,4841.000000,8555.000000,4293.000000
